In [158]:
import pandas as pd 
import numpy as np 
pd.set_option('display.max_columns',None)

In [159]:
def improt_data(lujing,code):
    data = pd.read_csv(lujing,encoding =code)
    return data

In [160]:
lujing = r'C:/Users/Administrator/Datas/爱尚花语/艾尚花语指标模型/test/clean_data.csv'
code = 'utf8'
data = improt_data(lujing,code)

In [161]:
data.head()

,Unnamed: 0,开始日期,商家ID,商家名称,运营组,省份,城市,是否新增商家,首次上线日期,门店在线状态,（日均）预设营业时长（h）,（日均）在线营业时长（h）,曝光人数,入店人数,下单人数,有效订单数,下单新客人数,复购人数,入店转化率,下单转化率,新客转化率,复购率,原价交易额,实付交易额,商家补贴金额,原价客单价,实付客单价,cluster
0,181,2,7392788,花生活鲜花（非凡花艺）,鲜花组,江苏省,淮安,否,20190714,在线,24,24.0,76589,1661,49,40,39,10,2.17,2.95,0.795918,0.204082,13908.0,7514.0,7944.0,283.836735,153.346939,0
1,28,2,3018522,花生活鲜花（源达花艺）,鲜花组,湖南省,常德,否,20170814,在线,17,17.8,63921,1134,35,31,30,5,1.77,3.09,0.857143,0.142857,8586.0,5139.8,4499.0,245.314286,146.851429,0
2,283,2,8197675,蔚然花坊,鲜花组,江苏省,南通,否,20191218,在线,14,14.9,63817,1084,5,2,2,3,1.70,0.46,0.400000,0.600000,1023.0,826.0,197.0,204.600000,165.200000,0
3,388,4,8755682,琪琪花店,鲜花组,广东省,佛山,否,20200324,在线,24,24.0,61978,1792,44,34,32,12,2.89,2.46,0.727273,0.272727,8159.0,4326.0,4913.0,185.431818,98.318182,0
4,112,2,6773940,花生活鲜花（鲜花婚庆）,鲜花组,山东省,德州,否,20190422,在线,23,23.6,57476,1216,7,6,6,1,2.12,0.58,0.857143,0.142857,1925.0,1135.0,985.0,275.000000,162.142857,0


In [162]:
data['复购人数'].isnull().count()

1594

In [182]:
rfm = data.groupby(['开始日期','商家ID','商家名称'],as_index=False).agg({'（日均）在线营业时长（h）': 'sum','下单人数':'sum', '实付交易额': 'sum'  })

In [185]:
rfm.columns = ['time','商家ID','商家名称','r','f','m']
rfm

,time,商家ID,商家名称,r,f,m
0,1,295680,花生活鲜花（鲜花格格花艺生活馆）,24.0,3,203.00
1,1,642989,花生活鲜花（春季花艺）,24.0,0,0.00
2,1,653643,花生活鲜花（雅景花艺）,10.4,2,347.00
3,1,714142,花生活鲜花（情意浓花店）,11.6,10,1373.09
4,1,720330,浪漫花屋,0.0,0,0.00
...,...,...,...,...,...,...
1589,4,8983876,熹媛,7.2,7,1205.00
1590,4,8983877,花管家鲜花批发,14.9,2,710.00
1591,4,8983878,鹿角花坊,13.2,0,0.00
1592,4,8985806,绿荫花店,14.8,0,0.00


In [186]:
desc_pd = rfm.iloc[:,2:].describe().T
desc_pd

,count,mean,std,min,25%,50%,75%,max
r,1594.0,15.508030,8.449015,0.0,10.125,17.3,24.0,24.0
f,1594.0,7.903388,16.773165,0.0,0.000,2.0,8.0,189.0
m,1594.0,990.427528,2454.651342,0.0,0.000,176.0,886.0,25703.0


In [187]:
r_bins = [-1,10.125,23,24]
f_bins = [-1,0,8,189]
m_bins = [-1,0,866,25703]

In [188]:
from sklearn.ensemble import RandomForestClassifier # RF库
clf = RandomForestClassifier()

In [189]:
rfm_merge = pd.merge(rfm,data,on='商家ID',how='inner')

In [190]:
clf = clf.fit(rfm_merge[['r','f','m']],rfm_merge['cluster'])
weights = clf.feature_importances_
print('feature importance:',weights)

feature importance: [0.34827781 0.20615613 0.44556606]


In [201]:
# RFM分箱得分
rfm['r_score'] = pd.cut(rfm['r'], r_bins, labels=[i+1 for i in range(len(r_bins)-1)])  # 计算R得分
rfm['f_score'] = pd.cut(rfm['f'], f_bins, labels=[i+1 for i in range(len(f_bins)-1)])  # 计算F得分
rfm['m_score'] = pd.cut(rfm['m'], m_bins, labels=[i+1 for i in range(len(m_bins)-1)])  # 计算M得分

In [202]:
# rfm_gb = rfm.apply(np.int32)

In [206]:
# 方法二：RFM组合
rfm_gb['r_score'] = rfm_gb['r_score'].astype(np.str)
rfm_gb['f_score'] = rfm_gb['f_score'].astype(np.str)
rfm_gb['m_score'] = rfm_gb['m_score'].astype(np.str)
rfm_gb['rfm_group'] = rfm_gb['r_score'].str.cat(rfm_gb['f_score']).str.cat(
rfm_gb['m_score'])

In [207]:
# rfm_gb.to_csv('clean_data_rfm.csv') 